In [1]:
import theano
import theano.tensor as T


Using gpu device 0: GeForce 940M


In [54]:
import keras

In [2]:
x = T.dmatrix('x')
s = 1 / (1 + T.exp(-x))
logistic = theano.function([x], s)
logistic([[0, 1], [-1, -2]])

array([[ 0.5       ,  0.73105858],
       [ 0.26894142,  0.11920292]])

In [52]:
x = T.scalar('x',dtype='float64')
compute = theano.function([x], 2*x);
compute(10)


array(20.0)

In [ ]:
np.distutils.__config__.show()

In [53]:
x= T.scalar('x',dtype='float64')
compute = theano.function([x], 2*x**2)
compute(2)


array(8.0)

In [12]:
x_value = theano.shared(10.0)
compute = theano.function([], 2*x , givens={x:x_value})
compute()





array(20.0)

In [55]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD


Using Theano backend.


In [57]:
model1 = Sequential()

In [ ]:
model1.add()